In [1]:
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from scipy import sparse, io
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Twitter

In [2]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="Pi")

In [3]:
train = sc.textFile("/user/data/Test_Train_set/train.csv")

In [5]:
x_train = list([x for x in train.map(lambda x:x[:-3]).toLocalIterator()])
y_train = list([x for x in train.map(lambda x:x[-1:]).toLocalIterator()])

In [6]:
twit = Twitter()
def tokenizer(text):
    return twit.morphs(text)

In [7]:
vect_1_3 = TfidfVectorizer(ngram_range=(1,5), min_df = 3, tokenizer=tokenizer).fit(x_train)

In [8]:
nx_train_1_3 = vect_1_3.transform(x_train)

In [11]:
param_grid_1_3 = {'C' : [1, 10.0, 100.0],'penalty' : ['l1','l2']}
grid_1_3 = GridSearchCV(LogisticRegression(), param_grid_1_3, cv=10) 
grid_1_3.fit(nx_train_1_3,y_train)

/home/vagrant/anaconda/lib/python3.5/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1, 10.0, 100.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [12]:
print(grid_1_3.best_score_) 
print(grid_1_3.best_params_)

0.901347324351
{'penalty': 'l2', 'C': 10.0}
